Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Import all necessary Libraries

In [156]:
import requests
import pandas as pd

#!conda install beautifulsoup4
!pip install beautifulsoup4
from bs4 import BeautifulSoup
#import lxml.html as lh

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import matplotlib.cm as cm # color library
import matplotlib.colors as colors # color library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


Assign Web page link to variable and request the page

In [157]:
#hint: requests.get()
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wikipedia_link)

Grab the page source code content using BeautifulSoup library

In [158]:
soup = BeautifulSoup(wiki_page.content)

Get the HTML corresponding the table of interest

In [159]:
tb = soup.find('table', class_='wikitable sortable')

Extract the table rows into a list

In [160]:
for tbl in tb:
    rows = tb.find_all('tr')
print(rows[0])

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>


Format list to be used for DF header: Remove HTML tags and extraenous characters from cell text. We then split the string to get a list.

In [161]:
str_cells = str(rows[0].find_all('th'))
cleantext = BeautifulSoup(str_cells).get_text()
cleantext = cleantext.replace(']', '') 
cleantext = cleantext.replace('[', '') 
cleantext = cleantext.replace('\n', '') 
cleantext = cleantext.replace(' ', '') 
lst_header = cleantext.split(',')
lst_header

['Postcode', 'Borough', 'Neighbourhood']

Format each row for DF: Remove HTML tags and extraenous characters from cell text. We then split the string to get a list.

In [162]:
new_rows = rows[1:]
list_rows = []
for row in new_rows:
    str_cells = str(row.find_all('td'))
    cleantext = BeautifulSoup(str_cells).get_text()
    cleantext = cleantext.replace(']', '') 
    cleantext = cleantext.replace('[', '') 
    cleantext = cleantext.replace('\n', '') 
    lst = cleantext.split(',')
    list_rows.append(lst)
print(list_rows[1])
type(list_rows)

['M2A', ' Not assigned', ' Not assigned']


list

Convert the list into a dataframe

In [163]:
df = pd.DataFrame(list_rows)
df.columns = lst_header
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Remove rows with " Not assigned" boroughs

In [164]:
df = df[df['Borough']!=" Not assigned"]
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Assigne borough's name to neighbourhood if neighbourhood not assigned

In [165]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == " Not assigned":
        row['Neighbourhood'] = row['Borough']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Using groupby to group borough and merge content of neighbourhood for borough with multiple neighbourhood

In [166]:
df_grouped = df.groupby(["Postcode", "Borough"])['Neighbourhood'].apply(lambda tags: ','.join(tags)).reset_index(name='Neighbourhood')
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Checking shape of Dataframe

In [167]:
df_grouped.shape

(103, 3)

Merge the Geo table with the original dataframe

In [168]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adding Geo data to main table

In [169]:
df_Table = pd.DataFrame.merge(df_grouped, df_geo, how='inner', left_on='Postcode', right_on='Postal Code', sort=True)
df_Table = df_Table.drop(columns=['Postal Code'])
df_Table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get Toronto geo coordinates

In [170]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Draw a map centered on Toronto

In [171]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Table['Latitude'], df_Table['Longitude'], df_Table['Borough'], df_Table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Set Foursquare account credentials

In [172]:
CLIENT_ID = 'FB2011MW0LW5YNMXRD1ZCSW3ZSH2AONBOFEW2FN2YK1Z321Y' # your Foursquare ID
CLIENT_SECRET = 'KISNJ11S43014YAZGQ30KT50ORWL03KMYCGNSUQTP0C01AGO' # your Foursquare Secret
VERSION = '20191023'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FB2011MW0LW5YNMXRD1ZCSW3ZSH2AONBOFEW2FN2YK1Z321Y
CLIENT_SECRET:KISNJ11S43014YAZGQ30KT50ORWL03KMYCGNSUQTP0C01AGO


Get name of Toronto's center neighbourhood

In [173]:
neighborhood_latitude = df_Table.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Table.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Table.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Rouge, Malvern are 43.806686299999996, -79.19435340000001.


Set foursquare url for area

In [174]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=FB2011MW0LW5YNMXRD1ZCSW3ZSH2AONBOFEW2FN2YK1Z321Y&client_secret=KISNJ11S43014YAZGQ30KT50ORWL03KMYCGNSUQTP0C01AGO&ll=43.653963,-79.387207&v=20191023&radius=500&limit=100'

extract json file from foursquare

In [175]:
results = requests.get(url).json()
#results

Create function that extracts the category of the venue

In [176]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [177]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Cafe Plenty,Café,43.654571,-79.389450
4,Rolltation,Japanese Restaurant,43.654918,-79.387424


Function to collect toronto venues info

In [178]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [179]:
# type your answer here

toronto_venues = getNearbyVenues(names=df_Table['Neighbourhood'],
                                   latitudes=df_Table['Latitude'],
                                   longitudes=df_Table['Longitude']
                                  )

 Rouge, Malvern
 Highland Creek, Rouge Hill, Port Union
 Guildwood, Morningside, West Hill
 Woburn
 Cedarbrae
 Scarborough Village
 East Birchmount Park, Ionview, Kennedy Park
 Clairlea, Golden Mile, Oakridge
 Cliffcrest, Cliffside, Scarborough Village West
 Birch Cliff, Cliffside West
 Dorset Park, Scarborough Town Centre, Wexford Heights
 Maryvale, Wexford
 Agincourt
 Clarks Corners, Sullivan, Tam O'Shanter
 Agincourt North, L'Amoreaux East, Milliken, Steeles East
 L'Amoreaux West
 Upper Rouge
 Hillcrest Village
 Fairview, Henry Farm, Oriole
 Bayview Village
 Silver Hills, York Mills
 Newtonbrook, Willowdale
 Willowdale South
 York Mills West
 Willowdale West
 Parkwoods
 Don Mills North
 Flemingdon Park, Don Mills South
 Bathurst Manor, Downsview North, Wilson Heights
 Northwood Park, York University
 CFB Toronto, Downsview East
 Downsview West
 Downsview Central
 Downsview Northwest
 Victoria Village
 Woodbine Gardens, Parkview Hill
 Woodbine Heights
 The Beaches
 Leaside
 Thornclif

Toronto venues table shape

In [180]:
print(toronto_venues.shape)
toronto_venues.head()

(2266, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Summary of neighbourhood count

In [181]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,6,6,6,6,6,6
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
Woburn,3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill",12,12,12,12,12,12


In [182]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


In [183]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
toronto_onehot.shape

(2266, 273)

Summary by categories

In [185]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, S...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
96,"Woodbine Gardens, Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.142857,0.0,0.0,0.00,0.0,0.00,0.0


Organize venues by popularity - trend value

In [186]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
  #  print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
   # print('\n')

---- Adelaide, King, Richmond----
---- Agincourt----
---- Agincourt North, L'Amoreaux East, Milliken, Steeles East----
---- Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
---- Alderwood, Long Branch----
---- Bathurst Manor, Downsview North, Wilson Heights----
---- Bayview Village----
---- Bedford Park, Lawrence Manor East----
---- Berczy Park----
---- Birch Cliff, Cliffside West----
---- Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe----
---- Brockton, Exhibition Place, Parkdale Village----
---- Business Reply Mail Processing Centre 969 Eastern----
---- CFB Toronto, Downsview East----
---- CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
---- Cabbagetown, St. James Town----
---- Caledonia-Fairbanks----
---- Canada Post Gateway Processing Centre----
---- Cedarbrae----
---- Central Bay Street----
---- Chinatown, Grange Park, Kensington Market----

In [187]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [188]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    Neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse,Sushi Restaurant,Hotel,Breakfast Spot,American Restaurant,Asian Restaurant
1,Agincourt,Lounge,Skating Rink,Sandwich Place,Chinese Restaurant,Print Shop,Breakfast Spot,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, S...",Playground,Asian Restaurant,Park,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate,...",Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Pizza Place,Gluten-free Restaurant,Deli / Bodega,Electronics Store
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Coffee Shop,Pool,Pub,Sandwich Place,Skating Rink,Gym,American Restaurant


Set up Clusters based on venues categories and popularity

In [189]:
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [190]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 6, 2, 1, 1, 6, 6, 6, 6, 6], dtype=int32)

In [191]:
# add clustering labels
Neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_Table

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(Neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].fillna(0).astype(int)

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Filipino Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,6,Pizza Place,Medical Center,Intersection,Rental Car Location,Electronics Store,Mexican Restaurant,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6,Hakka Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Caribbean Restaurant,Bank,Fried Chicken Joint,Discount Store,Doner Restaurant,Donut Shop


Map Toronto with cluster based color coded venues

In [192]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters